In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 5865, done.
remote: Counting objects: 100% (1054/1054), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 5865 (delta 896), reused 1035 (delta 879), pack-reused 4811
Receiving objects: 100% (5865/5865), 2.94 GiB | 59.91 MiB/s, done.
Resolving deltas: 100% (4722/4722), done.
Updating files: 100% (496/496), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 13.6 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/training-tuned-models/als_all.npz
/kaggle/input/training-tuned-models/__results__.html
/kaggle/input/training-tuned-models/ease_r_all.zip
/kaggle/input/training-tuned-models/mult_vae_2_train_validation.zip
/kaggle/input/training-tuned-models/als_xgboost_train_validation.npz
/kaggle/input/training-tuned-models/als_train_validation.npz
/kaggle/input/training-tuned-models/mult_vae_2_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/slim_recall_train_validation.zip
/kaggle/input/training-tuned-models/ease_r_train_validation.zip
/kaggle/input/training-tuned-models/__notebook__.ipynb
/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ials_train_validation.zip
/kaggle/input/training-tuned-models/mult_vae_2_all.zip
/kaggle/input/training-tuned-models/ease_r_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/__output__.json
/kaggle/input/training-tuned-models/ials_all.zip
/kaggle/input/training-tun

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_SVDpp_Cython, MatrixFactorization_AsySVD_Cython, MatrixFactorization_WARP_Cython
from Recommenders.MatrixFactorization.PyTorch.MF_MSE_PyTorch import MF_MSE_PyTorch
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

from Recommenders.Hybrid.RoundRobinListCombinationRecommender import RoundRobinListCombinationRecommender

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters_rp3': True,
    'tune_parameters_knn': True,
    'database_path': '/kaggle/working/tuning.db',
    'copy_prev_best_params': True,
    'save_github': True
}

In [10]:
shutil.copyfile(
    '/kaggle/input/training-tuned-models/slim_recall_train_validation.zip', 
    '/kaggle/working/slim_recall_train_validation.zip', 
)

'/kaggle/working/slim_recall_train_validation.zip'

In [11]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_recall.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [12]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [13]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [14]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [15]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[50])

def objective_function_rp3_recall(trial):           
    params = {
        'alpha': trial.suggest_float('alpha', 1e-5, 10, log=True),
        'beta': trial.suggest_float('beta', 1e-5, 10, log=True),
        'topK': trial.suggest_int('topK', 5, 2000, log=True),
        'implicit': True,
        'normalize_similarity': True
    }
        
    recommender_instance = RP3betaRecommender(URM_train)
    recommender_instance.fit(**params)
        
    result_df, _ = evaluator.evaluateRecommender(recommender_instance)
    return result_df.loc[50, "RECALL"]

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [16]:
if config['tune_parameters_rp3']:
    study = optuna.create_study(direction='maximize', study_name='tuning_recall_rp3',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_function_rp3_recall, n_trials=200)

[I 2023-12-24 17:33:11,729] Using an existing study with name 'tuning_recall_rp3' instead of creating a new one.


RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2597.20 column/sec. Elapsed time 8.60 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 7.16 sec. Users per second: 1461


[I 2023-12-24 17:33:30,776] Trial 200 finished with value: 0.27085447520659683 and parameters: {'alpha': 0.45212947578517376, 'beta': 0.15246559831421005, 'topK': 164}. Best is trial 158 with value: 0.27484076465819407.


RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2782.42 column/sec. Elapsed time 8.03 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 7.11 sec. Users per second: 1470


[I 2023-12-24 17:33:49,124] Trial 201 finished with value: 0.27305945703786 and parameters: {'alpha': 0.17501739096021174, 'beta': 0.19170695558429482, 'topK': 150}. Best is trial 158 with value: 0.27484076465819407.


...

[I 2023-12-24 18:41:56,997] Trial 397 finished with value: 0.22860380907302352 and parameters: {'alpha': 0.18558152391000024, 'beta': 0.5546256542880139, 'topK': 178}. Best is trial 202 with value: 0.2752977069192318.


RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 3412.75 column/sec. Elapsed time 6.55 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 6.99 sec. Users per second: 1495


[I 2023-12-24 18:42:12,618] Trial 398 finished with value: 0.2706947953867397 and parameters: {'alpha': 0.11972500608595396, 'beta': 0.28900720123865364, 'topK': 86}. Best is trial 202 with value: 0.2752977069192318.


RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2060.39 column/sec. Elapsed time 10.85 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 7.82 sec. Users per second: 1337


[I 2023-12-24 18:42:36,441] Trial 399 finished with value: 0.2677091689661575 and parameters: {'alpha': 8.608574256176213e-05, 'beta': 0.18705530295377992, 'topK': 288}. Best is trial 202 with value: 0.2752977069192318.


In [17]:
if config['tune_parameters_rp3']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [18]:
if config['tune_parameters_rp3']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [19]:
if config['tune_parameters_rp3']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [20]:
if config['tune_parameters_rp3']:
    with open('/kaggle/working/best_params_rp3_recall@50.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            '/kaggle/working/best_params_rp3_recall@50.json', 
            'tuning_results/best_params_rp3_recall@50.json', 
            'RP3 tuning results with recall@50(from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_rp3_recall@50.json', 
        '/kaggle/working/best_params_rp3_recall@50.json'
    )

In [21]:
def objective_function_knn_recall(trial):           
    params = {
        'topK': trial.suggest_int('topK', 10, 2000),
        'shrink': trial.suggest_int('shrink', 0, 500),
        'similarity': trial.suggest_categorical('similarity', ["cosine", "asymmetric", "dice", "jaccard", "tanimoto", "tversky", "euclidean"]),
        'feature_weighting': trial.suggest_categorical('feature_weighting', ["BM25", "TF-IDF", "none"])
    }
    
    if params['similarity'] == "asymmetric":
        params["asymmetric_alpha"] = trial.suggest_float("asymmetric_alpha", 0, 2, log=False)
        params["normalize"] = True     

    elif params['similarity'] == "tversky":
        params["tversky_alpha"] = trial.suggest_float("tversky_alpha", 0, 2, log=False)
        params["tversky_beta"] = trial.suggest_float("tversky_beta", 0, 5, log=False)
        params["normalize"] = True 

    elif params['similarity'] == "euclidean":
        params["normalize_avg_row"] = trial.suggest_categorical("normalize_avg_row", [True, False])
        params["similarity_from_distance_mode"] = trial.suggest_categorical("similarity_from_distance_mode", ["lin", "log", "exp"])
        params["normalize"] = trial.suggest_categorical("normalize", [True, False])
    
    recommender = ItemKNNCFRecommender(URM_train)
    recommender.fit(**params)
        
    result_df, _ = evaluator.evaluateRecommender(recommender)
    return result_df.loc[50, "RECALL"]

In [22]:
if config['tune_parameters_knn']:
    study = optuna.create_study(direction='maximize', study_name='tuning_recall_knn',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective_function_knn_recall, n_trials=200)

[I 2023-12-24 18:42:47,972] Using an existing study with name 'tuning_recall_knn' instead of creating a new one.


ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 6735.33 column/sec. Elapsed time 3.32 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 12.50 sec. Users per second: 837


[I 2023-12-24 18:43:04,317] Trial 200 finished with value: 0.2555008730857432 and parameters: {'topK': 207, 'shrink': 469, 'similarity': 'tanimoto', 'feature_weighting': 'BM25'}. Best is trial 171 with value: 0.25669798926621284.


ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 6399.24 column/sec. Elapsed time 3.49 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 11.61 sec. Users per second: 901


[I 2023-12-24 18:43:19,850] Trial 201 finished with value: 0.2566165916215631 and parameters: {'topK': 178, 'shrink': 461, 'similarity': 'tanimoto', 'feature_weighting': 'BM25'}. Best is trial 171 with value: 0.25669798926621284.


...

[I 2023-12-24 19:36:20,426] Trial 397 finished with value: 0.25949906521438154 and parameters: {'topK': 40, 'shrink': 486, 'similarity': 'tversky', 'feature_weighting': 'BM25', 'tversky_alpha': 0.289864672166714, 'tversky_beta': 1.038442725483388}. Best is trial 389 with value: 0.26011709750288536.


ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 6615.47 column/sec. Elapsed time 3.38 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 8.35 sec. Users per second: 1252


[I 2023-12-24 19:36:32,509] Trial 398 finished with value: 0.2556183269757614 and parameters: {'topK': 24, 'shrink': 500, 'similarity': 'tversky', 'feature_weighting': 'BM25', 'tversky_alpha': 0.2772424985055316, 'tversky_beta': 0.9514947093573511}. Best is trial 389 with value: 0.26011709750288536.


ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 6680.09 column/sec. Elapsed time 3.35 sec
EvaluatorHoldout: Processed 10456 (100.0%) in 9.63 sec. Users per second: 1086


[I 2023-12-24 19:36:45,824] Trial 399 finished with value: 0.2584001299129623 and parameters: {'topK': 46, 'shrink': 484, 'similarity': 'tversky', 'feature_weighting': 'BM25', 'tversky_alpha': 0.31662666683024193, 'tversky_beta': 0.8126779951794433}. Best is trial 389 with value: 0.26011709750288536.


In [23]:
if config['tune_parameters_knn']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [24]:
if config['tune_parameters_knn']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [25]:
if config['tune_parameters_knn']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [26]:
if config['tune_parameters_knn']:
    with open('/kaggle/working/best_params_item_knn_recall@50.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            '/kaggle/working/best_params_item_knn_recall@50.json', 
            'tuning_results/best_params_item_knn_recall@50.json', 
            'Item KNN tuning results with recall@50(from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_item_knn_recall@50.json', 
        '/kaggle/working/best_params_item_knn_recall@50.json'
    )

In [27]:
if config['save_github']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_recall.db',
        'Tuning db updated results (from kaggle notebook)'
    )

In [28]:
with open('/kaggle/working/best_params_rp3_recall@50.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3['implicit'] = True
params_rp3['normalize_similarity'] = True
params_rp3

{'alpha': 0.2139985499567939,
 'beta': 0.2194371742883361,
 'topK': 154,
 'implicit': True,
 'normalize_similarity': True}

In [29]:
with open('/kaggle/working/best_params_item_knn_recall@50.json', 'r') as params_file:
    params_knn = json.load(params_file)
params_knn

{'topK': 94,
 'shrink': 479,
 'similarity': 'tversky',
 'feature_weighting': 'BM25',
 'tversky_alpha': 0.010077514527591758,
 'tversky_beta': 1.6400961840323336}

In [30]:
slim = SLIMElasticNetRecommender(URM_train)
slim.load_model('/kaggle/working/', 'slim_recall_train_validation.zip')

rp3 = RP3betaRecommender(URM_train)
rp3.fit(**params_rp3)

knn = ItemKNNCFRecommender(URM_train)
knn.fit(**params_knn)

SLIMElasticNetRecommender: URM Detected 627 ( 4.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 247 ( 1.1%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file '/kaggle/working/slim_recall_train_validation.zip'
SLIMElasticNetRecommender: Loading complete
RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2677.97 column/sec. Elapsed time 8.34 sec
ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 6565.51 column/sec. Elapsed time 3.40 sec


In [31]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10, 50, 100])

result_df, _ = evaluator.evaluateRecommender(slim)
display(result_df)
result_df, _ = evaluator.evaluateRecommender(rp3)
display(result_df)
result_df, _ = evaluator.evaluateRecommender(knn)
display(result_df)

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 20.55 sec. Users per second: 509


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.090838,0.159452,0.135748,0.046138,0.08011,0.266824,0.145528,0.108842,0.513676,0.347613,...,0.802826,0.412393,0.802826,0.025451,9.136736,0.99532,0.073435,0.703859,2.491186,0.277515
50,0.04328,0.276664,0.275857,0.013338,0.083426,0.276563,0.18455,0.074822,0.703615,0.403729,...,0.802826,0.56488,0.802826,0.056744,10.29789,0.998208,0.163729,0.793309,1.831304,1.460054
100,0.029097,0.346429,0.346349,0.007379,0.08698,0.277285,0.206773,0.053684,0.75306,0.414268,...,0.802826,0.604576,0.802826,0.086735,10.927168,0.99905,0.250266,0.841786,1.478013,3.01804


EvaluatorHoldout: Processed 10456 (100.0%) in 14.85 sec. Users per second: 704


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.088227,0.156875,0.133564,0.042917,0.075914,0.249399,0.139358,0.106262,0.508225,0.323788,...,0.802826,0.408016,0.802826,0.054732,9.60608,0.994165,0.157924,0.740015,2.499054,0.288198
50,0.043747,0.276135,0.27527,0.012928,0.080218,0.259436,0.180421,0.075496,0.703041,0.382018,...,0.802826,0.56442,0.802826,0.151269,11.323067,0.998756,0.436471,0.872285,1.529838,1.566582
100,0.029328,0.339721,0.339623,0.007227,0.083738,0.260032,0.201471,0.053993,0.744357,0.392603,...,0.802826,0.597589,0.802826,0.231205,12.10715,0.999538,0.667118,0.932688,1.145201,3.290909


EvaluatorHoldout: Processed 10456 (100.0%) in 19.90 sec. Users per second: 525


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.085023,0.149918,0.127434,0.041901,0.07372,0.245796,0.135182,0.101995,0.495409,0.317617,...,0.802826,0.397727,0.802826,0.044316,9.649551,0.995756,0.127868,0.743364,2.301399,0.287872
50,0.041249,0.260959,0.260117,0.012294,0.077034,0.255962,0.172997,0.071206,0.687835,0.372015,...,0.802826,0.552211,0.802826,0.094525,10.793667,0.998558,0.272743,0.831502,1.670169,1.509895
100,0.028028,0.331003,0.330913,0.00684,0.080486,0.256762,0.195234,0.051679,0.741775,0.382531,...,0.802826,0.595516,0.802826,0.129593,11.326145,0.999208,0.373928,0.872522,1.380521,3.105441


In [32]:
slim_rp3 = RoundRobinListCombinationRecommender(URM_train, [slim, rp3])
slim_knn = RoundRobinListCombinationRecommender(URM_train, [slim, knn])
rp3_knn = RoundRobinListCombinationRecommender(URM_train, [knn, rp3])
all_ = RoundRobinListCombinationRecommender(URM_train, [slim, rp3, knn])

result_df, _ = evaluator.evaluateRecommender(slim_rp3)
display(result_df)
result_df, _ = evaluator.evaluateRecommender(slim_knn)
display(result_df)
result_df, _ = evaluator.evaluateRecommender(rp3_knn)
display(result_df)
result_df, _ = evaluator.evaluateRecommender(all_)
display(result_df)

EvaluatorHoldout: Processed 10456 (100.0%) in 27.98 sec. Users per second: 374


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.091421,0.16092,0.136818,0.04597,0.079996,0.266776,0.145872,0.109605,0.517693,0.346541,...,0.802826,0.415617,0.802826,0.036731,9.273591,0.994268,0.105983,0.714401,2.585797,0.280993
50,0.044422,0.280708,0.279841,0.013616,0.084032,0.276584,0.186522,0.076673,0.706197,0.405215,...,0.802826,0.566953,0.802826,0.099827,10.752855,0.998331,0.28804,0.828358,1.754775,1.504015
100,0.029869,0.351864,0.351774,0.007575,0.087764,0.277324,0.209223,0.055063,0.756312,0.416108,...,0.802826,0.607187,0.802826,0.154052,11.46666,0.999201,0.444503,0.883347,1.386098,3.130864


EvaluatorHoldout: Processed 10456 (100.0%) in 30.81 sec. Users per second: 339


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.090895,0.159764,0.135784,0.045881,0.079782,0.267164,0.145442,0.108895,0.516832,0.346453,...,0.802826,0.414926,0.802826,0.03104,9.250595,0.995011,0.089564,0.71263,2.500783,0.280146
50,0.043305,0.277525,0.276681,0.013366,0.083257,0.276921,0.184674,0.074889,0.706676,0.402987,...,0.802826,0.567337,0.802826,0.066412,10.364305,0.9981,0.191625,0.798426,1.865726,1.467912
100,0.029141,0.349888,0.349799,0.0074,0.086821,0.277684,0.207347,0.0538,0.759181,0.413606,...,0.802826,0.60949,0.802826,0.095589,10.925708,0.998931,0.275812,0.841674,1.540084,3.020869


EvaluatorHoldout: Processed 10456 (100.0%) in 25.15 sec. Users per second: 416


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.08795,0.157331,0.134243,0.042825,0.075681,0.250079,0.139374,0.106274,0.510042,0.323852,...,0.802826,0.409475,0.802826,0.045317,9.4612,0.994123,0.130758,0.728854,2.539847,0.285164
50,0.043248,0.274491,0.27362,0.012793,0.079827,0.260109,0.179741,0.07469,0.702181,0.381198,...,0.802826,0.563729,0.802826,0.112388,10.885833,0.998392,0.324286,0.838602,1.712867,1.520038
100,0.029274,0.347108,0.347015,0.007135,0.083508,0.260848,0.202713,0.053993,0.752678,0.392039,...,0.802826,0.604269,0.802826,0.165771,11.553596,0.99922,0.478316,0.890044,1.36437,3.155733


EvaluatorHoldout: Processed 10456 (100.0%) in 36.41 sec. Users per second: 287


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.090953,0.160955,0.137169,0.045622,0.079604,0.266665,0.145532,0.109379,0.519223,0.345234,...,0.802826,0.416846,0.802826,0.035778,9.250109,0.994158,0.103234,0.712592,2.600796,0.280658
50,0.043984,0.280038,0.279173,0.013463,0.083513,0.276453,0.185788,0.075995,0.708684,0.403253,...,0.802826,0.56895,0.802826,0.088902,10.590018,0.998145,0.256517,0.815814,1.828649,1.48942
100,0.029699,0.352665,0.352576,0.007488,0.087245,0.277177,0.208837,0.054783,0.758512,0.414192,...,0.802826,0.608953,0.802826,0.133536,11.241654,0.999036,0.385305,0.866013,1.476777,3.085695
